In [4]:
import requests
import cv2
import glob
import os, sys
url = "https://kapi.kakao.com/v1/vision/face/detect"
MYAPI_KEY = '5a826bf53f8508d2cbf62d2199a967dd'

In [5]:
def face_detector(filename):
    headers = {'Authorization': 'KakaoAK {}'.format(MYAPI_KEY)}
    try:
        files = {'file' : open(filename, 'rb')}
        resp = requests.post(url, headers=headers, files=files)
        resp.raise_for_status()
        return resp.json()
    except Exception as e:
        print(str(e))
        sys.exit(0)

In [6]:
webcam = cv2.VideoCapture(0) 
while (webcam.isOpened()) :
    ret, frame = webcam.read()
    #print("ret", ret, "\nframe", frame)    
    if ret :
        cv2.imwrite('capture.png',frame) # 이미지저장 
        #print("frame shape", frame.shape)
        height, width, d = frame.shape
        #print("height", height, "\twidth", width, "\td", d)
        files = glob.glob('*.png')
        for file in files:
            json_data = face_detector(file)
            #print('json data', json_data)
            # ========== write all facial info in text file ==========
            with open("json_response_webcam.txt", "w") as text_file:
                text_file.write(str(json_data))
            face =  json_data["result"]["faces"][0]
            #print('face', face)
            x = int(face['x'] * width)
            y = int(face['y'] * height)
            w = int(face['w'] * width)
            h = int(face['h'] * height)
            #print("x", x, "\ty", y, "\tw", w, "\h", h)
            # ========== facial points like right eye, left eye, etc. ==========
            facial_points = face['facial_points'] 
            # ========== gender, age ==========
            facial_attr = face['facial_attributes'] 
            #print('\n', facial_attr)
            gender = facial_attr['gender']
            #print('\n', gender, gender['male'], gender['female'])
            if (gender['male'] > gender['female']) :
                gender = 'male'
            else :
                gender = 'female'
            #print('GENDER :', gender)
            age = int(facial_attr['age'])
            #print('AGE :', age)
            # ========== draw rectangle & resize frame ========== 
            cv2.rectangle(frame,(x,y),((x+w),(y+h)),(255, 0, 0),3)
            resize_frame = cv2.resize(frame, dsize=(640, 480), interpolation=cv2.INTER_AREA)
            # ========== get score of similarity & facial landmarks ========== 
            score = face['score']*100
            print('score', score)
            target_obj = ['right_eyebrow', 'left_eyebrow', 'jaw', 'right_eye', 'left_eye', 'nose', 'lip']
            fig_w, fig_h = json_data['result']['width'], json_data['result']['height']
            for each_obj in target_obj:
                for each in facial_points[each_obj]:
                    cv2.circle(img=resize_frame, center=(int(each[0]*fig_w), int(each[1]*fig_h)), radius=3, color=(0, 255, 0), thickness=-1)
                    if 80 <= score: 
                        answer = "Linda {:.2f}%".format(score)
                        cv2.putText(resize_frame, answer, (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    else : 
                        cv2.putText(resize_frame, 'not recognized', (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        # ========== Display the resulting frame ========== 
        cv2.imshow('KAKAO VISION API EXAMPLE', resize_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
webcam.release()
cv2.destroyAllWindows()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



score 99.26740527153015
400 Client Error: Bad Request for url: https://kapi.kakao.com/v1/vision/face/detect
Traceback (most recent call last):
  File "<ipython-input-5-76426d23cb8b>", line 6, in face_detector
    resp.raise_for_status()
  File "C:\Users\82109\anaconda3\lib\site-packages\requests\models.py", line 941, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://kapi.kakao.com/v1/vision/face/detect

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\82109\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-79549a3649a2>", line 12, in <module>
    json_data = face_detector(file)
  File "<ipython-input-5-76426d23cb8b>", line 10, in face_detector
    sys.exit(0)
SystemExit: 0

During handling of the above ex

TypeError: object of type 'NoneType' has no len()